## [Link para documento overleaf](https://www.overleaf.com/project/677fe7fd4bf7cc5092992e57) 

In [44]:
#Bibliotecas
import pandas as pd
from typing import List,Callable

In [65]:
#Funções Intermédiarias da Função Principal
def extrair_informacoes() -> pd.DataFrame:
    '''
    Inputs:
        None
    Output:
        inf_fundos: DataFrame com Patrimônio Líquido, Range Mínimo e Máximo de Crédito para cada fundo ['fundo','PL','l_min','l_max']
        book_fundos: DataFrame com Percentual de Alocação de cada Book (Esse perccentual é referente a parte disponivel para crédito) para cada fundo ['fundo','book']
    '''
    

def calculo_regua(inf_fundos: pd.DataFrame, books_fundos:pd.DataFrame, credito_total:float) -> pd.DataFrame:
    '''
    Inputs:
        inf_fundos: DataFrame com Patrimônio Líquido, Range Mínimo e Máximo de Crédito para cada fundo ['fundo','PL','l_min','l_max']
        books_fundos: DataFrame com Percentual de Alocação de cada Book (Esse perccentual é referente a parte disponivel para crédito) para cada fundo ['fundo','books']
        credito_total: float com o valor total do crédito 
    Output:
        regua: DataFrame com as informações da régua que contém as alocações ideais ['fundo','book','percentual_alocacao']
    '''

    #Funções Auxiliares
    def percentual_alocacao_range_total(credito_total:float,inf_fundos: pd.DataFrame) -> float:
        inf_fundos['v_min'] = inf_fundos['PL'] * inf_fundos['l_min']
        inf_fundos['v_max'] = inf_fundos['PL'] * inf_fundos['l_max']
        return (credito_total - inf_fundos['v_min'].sum())/ (inf_fundos['v_max'].sum() - inf_fundos['v_min'].sum())
    
    def percentual_credito_relativo_pl(percentual_alocacao_range_total:float,inf_fundo: pd.Series) -> dict:
        return inf_fundo['l_min'] + percentual_alocacao_range_total * (inf_fundo['l_max'] - inf_fundo['l_min'])
    
    def percentual_book_relativo_pl(percentual_credito_relativo_pl:float,peso_book_fundo: float) -> float:
        return percentual_credito_relativo_pl * peso_book_fundo
    
    def valor_book_relativo_pl(percentual_book_relativo_pl:float,pl:float) -> float:
        return pl *percentual_book_relativo_pl

    #Cálculo da Régua
    percentual_alocacao_range_total = percentual_alocacao_range_total(credito_total,inf_fundos)
    
    percentuais_credito_relativo_pl = {}
    for index,row in inf_fundos.iterrows():
        percentuais_credito_relativo_pl[row['fundo']] = percentual_credito_relativo_pl(percentual_alocacao_range_total,row)

    percentuais_books_relativo_pl = pd.DataFrame(columns=['fundo','book','percentual'])
    for fundo,percentual_credito_relativo_pl in percentuais_credito_relativo_pl.items():
        for index,row in books_fundos[books_fundos['fundo'] == fundo].iterrows():
            percentuais_books_relativo_pl.loc[len(percentuais_books_relativo_pl)] = [fundo,row['book'],percentual_book_relativo_pl(percentual_credito_relativo_pl,row['peso_book'])]
    
    valores_books_relativos_pl = pd.DataFrame(columns=['fundo','book','valor'])
    for index,row in percentuais_books_relativo_pl.iterrows():
        valores_books_relativos_pl.loc[len(valores_books_relativos_pl)] = [row['fundo'],row['book'],valor_book_relativo_pl(row['percentual'],inf_fundos[inf_fundos['fundo'] == row['fundo']]['PL'].values[0])]
   
    regua = pd.DataFrame(columns=['fundo','book','percentual_alocacao'])
    books = valores_books_relativos_pl['book'].unique()
    fundos = valores_books_relativos_pl['fundo'].unique()
    somas_books = {}
    for book in books:
        somas_books[book] = valores_books_relativos_pl[valores_books_relativos_pl['book'] == book]['valor'].sum()
    for fundo in fundos:
        for index,row in valores_books_relativos_pl[valores_books_relativos_pl['fundo'] == fundo].iterrows():
            regua.loc[len(regua)] = [fundo,row['book'],row['valor']/somas_books[row['book']]]
    
    return regua

In [ ]:
#Testes Funções Intermédiarias

#Teste Cálculo Régua - Exemplo feito pelo Ali na Reunião do dia 09/01/2025 (https://drive.google.com/file/d/1O-bVrF1At0xZNQIGmIOQ1Oi7YiSkGD0u/view?usp=sharing)
credito_total = 40
teste_inf_fundos = pd.DataFrame({
        'fundo':['RFA','APO','ID2'],
        'PL':[100,50,70],
        'l_min':[0.1,0.1,0.1],
        'l_max':[0.2,0.2,0.4]
})
teste_books_fundos  = pd.DataFrame({
        'fundo':['RFA','RFA','RFA','APO','APO','APO','ID2','ID2','ID2'],
        'book':['HG','MY','HY','HG','MY','HY','HG','MY','HY'],
        'peso_book':[0.53778,0.1634,0.29882,0.23006,0.433212,0.336729,0.41028,0.20813,0.38159]
})
#print(calculo_regua(teste_inf_fundos,teste_books_fundos,credito_total)) -> Funcionando







  fundo book  percentual_alocacao
0   RFA   HG             0.475293
1   RFA   MY             0.262353
2   RFA   HY             0.327520
3   APO   HG             0.101664
4   APO   MY             0.347780
5   APO   HY             0.184535
6   ID2   HG             0.423043
7   ID2   MY             0.389866
8   ID2   HY             0.487946


In [0]:
#Função Principal
def solucao(checagens: List[Callable],ordens: pd.DataFrame) -> pd.DataFrame:
    '''
    Função principal que recebe como input:
    1. checagens: Lista de funções geradas pelo ChatGPT
    2. ordens: DataFrame ['Ticker','Amount','Price'] representando uma grupo de ordens a serem boletadas
    e retorna:
    1. ordens_final: DatFrame ['Ticker','Amount','Price','Fundo'] representando quanto irá para cada fundo
    '''
    #1.Etapa - Extrair Informações dos Fundos (Patrimônio Líquido, Range Mínimo e Máximo de Crédito, Pesos de cada Book em cada Fundo)
    inf_fundos = pd.DataFrame()
    book_fundos = pd.DataFrame()
    #2Etapa -> Calcular a Régua
    regua = calculo_regua(inf_fundos,books_fundos,credito_total)
   

{'RFA': [6165720738.95119], 'APO': [2279627646.384785], 'ID2': [3488179932.21023], 'APP': [8430325498.580385], 'PID': [3937179801.4903502], 'KAT': [3004467197.6176753], 'KCP': [3424900701.5180855], 'CPI': [54355074.65991], 'BVP': [1191376868.3027952], 'PAL': [18720368.46087], 'IRF': [168954523.486945], '846': [3130395619.174485], '134': [7003331463.740895], '678': [6318358039.8425455], 'KOP': [733760002.8074601], 'FRA': [78590201.440485], 'PDA': [1172652382.054455], 'KRF': [2061515384.744035], '652': [2496049914.0221553], '389': [94841789.981635]}
